In [4]:
import visualize_prediction as V
import eval_model as E
import pandas as pd
from torchvision import transforms, utils

#suppress pytorch warnings about source code changes
import warnings
warnings.filterwarnings('ignore')

## Loading dataset

In [2]:
PATH_TO_IMAGES = "dataset/"
PATH_TO_MODEL = "pretrained/checkpoint"

In [6]:
dataloader,model= V.load_rsna_data(PATH_TO_IMAGES,PATH_TO_MODEL,fold='val')
print(model)
print(len(dataloader))

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
    (relu0): ReLU(inplace)
    (pool0): MaxPool2d(kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), dilation=(1, 1), ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm.1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm.2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True)
        (relu.2): ReLU(inplace)
        (conv.2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm.1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True)
        (relu.1): ReLU(inplace)
        (conv.1): Conv2d(96, 128, kernel_size=(1, 1), stride=(1, 1), bias

## Applying the prediction model

In [ ]:
LABEL='Pneumonia'
model=model.cpu()
preds = V.show_next(dataloader, model, LABEL)
preds

## Testing DenseNet121 pretrained in the ChestXray14 in the RSNA dataset

In [ ]:
# use imagenet mean,std for normalization
mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomHorizontalFlip(),
        transforms.Scale(224),
        # because scale doesn't always give 224 x 224, this ensures 224 x
        # 224
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
    'val': transforms.Compose([
        transforms.Scale(224),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean, std)
    ]),
}

model = model.cuda()
preds, aucs = E.make_pred_multilabel(
        data_transforms, model, PATH_TO_IMAGES)
print(preds)
print(aucs)